# 데일리차트 접속 후 인기 포메이션, 인기 팀 컬러 조사하기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_autoinstaller
import sys
import os
import re
import requests
import pandas as pd
import numpy as np
import time

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
url = 'https://fifaonline4.nexon.com/datacenter/dailysquad'
driver.get(url)
response = requests.get(url)
html = response.text

C:\Users\user\AppData\Local\Temp/ipykernel_8552/2310303632.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [2]:
contents = driver.find_elements(By.CSS_SELECTOR,'#formationSwiper > div > div > div > div')
len(contents)

10

In [3]:
contents[0].find_element(By.CSS_SELECTOR,'div.txt').text # 잘 나오는 것 확인

'4-2-3-1'

In [4]:
formation = []
for content in contents:
    formae = content.find_element(By.CSS_SELECTOR,'div.txt').text
    formation.append(formae)
    
formation_top6 = formation[:6] # 상위 6개 포메이션 선정
print(formation_top6)

['4-2-3-1', '4-2-2-1-1', '5-2-3', '4-2-2-2', '4-2-1-3', '5-2-1-2']


In [5]:
teams = driver.find_elements(By.CSS_SELECTOR,'#divTeamColorInfo > div')
print(len(teams))

10


In [6]:
print(teams[0].find_element(By.CSS_SELECTOR,'div.txt').text)

대한민국


In [7]:
team_color=[]

for team in teams:
    tc = team.find_element(By.CSS_SELECTOR,'div.txt').text
    team_color.append(tc)
    
team_color_top4 = team_color[:4] # 상위 4개 팀컬러 선정
print(team_color_top4)

['대한민국', '첼시', '레알 마드리드', '바이에른 뮌헨']


In [8]:
driver.quit()

In [9]:
PlayersData = pd.read_csv("./players_data/players_data.csv",encoding='cp949') # 추출한 선수 명단 데이터를 불러온다.

In [10]:
PlayersData

,포지션,이름,클래스
0,ST,박주영,2012KH
1,ST,R. 레반도프스키,22TOTY
2,ST,크리스티아누 호날두,22TOTS
3,ST,호나우두,MC
4,ST,크리스티아누 호날두,21TOTY
...,...,...,...
235,GK,에우통 레이치,21UCL
236,GK,닉 포프,LIVE
237,GK,조 하트,TC
238,GK,보이치에흐 슈체스니,EBS


In [11]:
PlayersData_list = PlayersData.values.tolist()
print(type(PlayersData_list))
print(PlayersData_list)

<class 'list'>
[['ST', '박주영', '2012KH'], ['ST', 'R. 레반도프스키', '22TOTY'], ['ST', '크리스티아누 호날두', '22TOTS'], ['ST', '호나우두', 'MC'], ['ST', '크리스티아누 호날두', '21TOTY'], ['ST', '손흥민', '22TOTY'], ['ST', 'R. 레반도프스키', '21TOTY'], ['ST', '크리스티아누 호날두', 'MC'], ['ST', '지동원', '2012KH'], ['ST', '사뮈엘 에토', 'FA'], ['ST', '차범근', 'LN'], ['ST', '카이 하베르츠', '21UCL'], ['ST', 'R. 레반도프스키', '22TOTS'], ['ST', '손흥민', '21TOTY'], ['ST', '크리스티아누 호날두', 'LH'], ['ST', '차범근', 'LH'], ['CF', '박주영', '2012KH'], ['CF', '지동원', '2012KH'], ['CF', '사디오 마네', '21UCL'], ['CF', '이반 페리시치', 'LOL'], ['CF', '차범근', 'LH'], ['CF', '세르주 그나브리', '21UCL'], ['CF', '카이 하베르츠', '21UCL'], ['CF', '손흥민', '22TOTY'], ['CF', '고정운', 'TKL'], ['CF', '이반 페리시치', '21UCL'], ['CF', '카림 벤제마', 'BOE21'], ['CF', '메이슨 마운트', '22TOTN'], ['CF', '호나우두', 'MC'], ['CF', '손흥민', '21TOTY'], ['CF', '카림 벤제마', '21UCL'], ['CF', '모하메드 살라', '22TOTS'], ['LW', '사디오 마네', '21UCL'], ['LW', '에덴 아자르', 'EBS'], ['LW', '고정운', 'TKL'], ['LW', '필 포든', '22TOTS'], ['LW', '차범근', 'LH'], ['LW', '비니시우스 주니어',

# 스쿼드메이커를 이용한 자동 스쿼드 제작 시스템

## 프로세스

1. 인기 포메이션 6개 중 하나를 선정한다.
2. 선정한 상위 4개의 팀 컬러 중 하나를 팀 컬러 검색창에 입력
3. 전제조건

    1) 포지션별로 선수 입력 시 PlayersData에 있는 선수들을 먼저 입력한다. 단 포지션별로 반영할 데이터의 범위는 자체적으로 선정한다.
        - ex) 스쿼드메이커에서 LS,RS,ST,LF,RF,CF 자리는 PlayersData의 포지션이 ST,CF인 선수들 중에서 우선적으로 검색할 예정
        - 검색을 모두 완료하였음에도 없는 경우에는 공란으로 우선 남겨둔다.
    
    2) 선수의 포지션은 선수 카드에 주어진 포지션이 아니라 유저들이 사용한 포지션을 따른다.
   
        - ex) TKL 차두리의 원래 포지션은 RB이지만, 유저들은 CB으로 주로 사용하므로 TKL 차두리는 CB로 분류, PlayersData에도 반영되어 있다
        
    3) 스쿼드 작성 순서는 공격진부터 골키퍼 순으로 작성
    
        - 일반적으로 공격진부터 스쿼드를 짜기 때문이다.
    
    
4. PlayersData에 해당 선수가 존재하는 경우, 해당 선수의 이름과 시즌이 일치하는지 확인한 후 스쿼드에 포함

    - 이름 검색 시 여러 선수가 검색되는 경우도 있으므로(페드로 검색 시 페드로 포로도 함께 검색되는 것처럼) 이름, 시즌 전부 확인
    - 포지션별 인기 순으로 정렬된 것이므로 PlayersData 위에서부터 선수 이름 검색해서, 해당 선수가 존재하면 포함하면 된다.
    
5. 스쿼드 완료 후 급여가 초과된 경우 골키퍼, 측면 수비수 2명 중 급여가 높은 순으로 선수를 삭제한다.

    - 삭제 후 다시 급여가 남게 되면, 남은 급여에 맞춰서 다른 선수를 포함한다.
    
6. 팀 컬러는 클럽 컬러, 국가 컬러만 고려한다.

In [12]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path) 
url = 'https://fifaonline4.nexon.com/datacenter/squadmaker'
driver.get(url)

C:\Users\user\AppData\Local\Temp/ipykernel_8552/304761245.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


## FO4 모든 클럽 팀컬러, 국가 팀컬러 각각 리스트로 수집

In [13]:
squad = driver.find_element(By.CSS_SELECTOR,'#formationPlayer11')
squad.click()
time.sleep(2)
wrap_teamcolor = '#squadSearchform1 > div > div.search_form > div > div.search_input_detail > div.search_input_detail_teamcolor'
search_teamcolor = driver.find_element(By.CSS_SELECTOR, wrap_teamcolor+'>div.wrap_teamcolor')

club_teamcolor = ' div > ul > li:nth-child(2)'
search_teamcolor.click()
time.sleep(2)
club_searching = search_teamcolor.find_element(By.CSS_SELECTOR,club_teamcolor) # 클럽 선택
club_searching.click()
time.sleep(2)
club_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')


total_ClubColorList = [] # 피온4 모든 클럽 팀컬러, 찾고자 하는 팀컬러가 여기에 없으면 국가 팀컬러라고 전제한다.
for c in range(len(club_colorlists)):
    club = club_colorlists[c].text
    total_ClubColorList.append(club)
while '' in total_ClubColorList:
    total_ClubColorList.remove('')
total_ClubColorList = total_ClubColorList[1:]    

total_ClubColorList[:20]

['맨체스터 시티',
 '리버풀',
 '토트넘 홋스퍼',
 '첼시',
 '아스널',
 '맨체스터 유나이티드',
 '울버햄프턴 원더러스',
 '레스터 시티',
 '에버턴',
 '왓퍼드',
 '바이에른 뮌헨',
 '보루시아 도르트문트',
 'RB 라이프치히',
 '아인트라흐트 프랑크푸르트',
 '바이엘 04 레버쿠젠',
 'VfL 볼프스부르크',
 'FC 바르셀로나',
 '아틀레티코 마드리드',
 '레알 마드리드',
 '발렌시아 CF']

In [14]:
driver.quit()

## 로그인 진행(스쿼드메이커 저장 작업 시 로그인이 되어있어야 함)

In [15]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path) 
url = 'https://fifaonline4.nexon.com/datacenter/squadmaker'
driver.get(url)

C:\Users\user\AppData\Local\Temp/ipykernel_8552/304761245.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [16]:
login_btn = driver.find_element(By.CSS_SELECTOR,'body > div.gnbWrapper.gnbSizeL > div.gnbBar > div.gnbBarRight > div.gnbLogin').click()
alert = driver.switch_to.alert 
alert.accept() # 경고창 뜨는 거 방지

ID_input = driver.find_element(By.CSS_SELECTOR,'#txtNexonID').send_keys('khpoptop@gmail.com')
pwd_input = driver.find_element(By.CSS_SELECTOR,'#txtPWD').send_keys('jimkwon950615')
time.sleep(1)
click_loginbtn = driver.find_element(By.CSS_SELECTOR,'#nexonLogin > fieldset > div.btLogin > button').click()

### 포지션 정하기
> PlayersData 상에 있는 포지션

1. LS,RS,ST,LF,RF,CF -> PlayersData의 ST,CF에서 검색
    - Forward
2. LW,RW,LM,RM,LAM,RAM -> PlayersData의 LW,RW,LM,RM에서 검색
    - Winger
3. CAM -> PlayersData의 CAM에서 검색
    - Attack_mid
4. LCM,RCM,CM -> PlayersData의 CM,CAM에서 검색
    - Central_mid
5. LDM,RDM,DM -> PlayersData의 CDM,CM에서 검색
    - Defend_mid
6. LB,LWB -> PlayersData의 LB,LWB에서 검색
    - Leftback
7. RB,RWB -> PlayersData의 RB,RWB에서 검색
    - Rightback
8. LCB,RCB,CB -> PlayersData의 CB에서 검색
    - Centerback
9. GK는 GK에서 검색
    - Goalkeeper

In [17]:
"""def sort_position():
    PlayersData_sorted=[]
    for playersdata in PlayersData_list: # 선수 데이터 리스트
        if position_name=='LS' or position_name=='RS' or position_name=='ST' or position_name=='LF' or position_name=='RF' or position_name=='CF':
            if playersdata[0]=='ST' or playersdata[0]=='CF':
                PlayersData_sorted.append(playersdata)
        elif position_name=='LW' or position_name=='RW' or position_name=='LM' or position_name=='RM' or position_name=='LAM' or position_name=='RAM':
            if playersdata[0]=='LW' or playersdata[0]=='RW' or playersdata[0]=='LM' or playersdata[0]=='RM':
                PlayersData_sorted.append(playersdata)
        elif position_name=='CAM':
            if playersdata[0]=='CAM':
                PlayersData_sorted.append(playersdata)
        elif position_name == 'LCM' or position_name=='RCM' or position_name=='CM':
            if playersdata[0]=='CM' or playersdata[0]=='CAM':
                PlayersData_sorted.append(playersdata)
        elif position_name=='LDM' or position_name=='RDM' or position_name=='CDM':
            if playersdata[0]=='CDM' or playersdata[0]=='CM':
                PlayersData_sorted.append(playersdata)
        elif position_name=='LB' or position_name=='LWB':
            if playersdata[0]=='LB' or playersdata[0]=='LWB':
                PlayersData_sorted.append(playersdata)
        elif position_name=='RB' or position_name=='RWB':
            if playersdata[0]=='RB' or playersdata[0]=='RWB':
                PlayersData_sorted.append(playersdata)
        elif position_name=='LCB' or position_name=='RCB' or position_name=='CB':
            if playersdata[0]=='CB':
                PlayersData_sorted.append(playersdata)
        elif position_name=='GK':
            if playersdata[0]=='GK':
                PlayersData_sorted.append(playersdata)
        else:
            continue"""

In [45]:
def choose_player():   
    for player in PlayersData_sorted:
        player_input = driver.find_element(By.CSS_SELECTOR,'#searchName').send_keys(player[1])
        time.sleep(1)
        player_searchbtn = driver.find_element(By.CSS_SELECTOR,'#squadSearchPlayerBtn').click()
        time.sleep(2)
        search_result = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li > div > div.info_top > div.name') # 검색 결과
        #checked = driver.find_element(By.XPATH,'//*[@id="checkbox1"]').get_attribute('checked') # 스쿼드에 미리 포함되어있는지 확인
        if len(search_result)==0 or driver.find_element(By.XPATH,'//*[@id="checkbox1"]').get_attribute('checked')=='true':
            continue
        else:
            player_name = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li > div > div.info_top > div.name')[0].text
            player_searched = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li') # 검색한 선수 목록
            for k in range(len(player_searched)):
                divTag = driver.find_element(By.CSS_SELECTOR,'#ulPlayerList > li.player.player'+str(k)+' > div > div.info_top > div.season > img')
                src = divTag.get_attribute('src')
                regex = re.compile('{}(.*){}'.format(re.escape('season/'), re.escape('.png')))
                player_season = regex.findall(str(src))[0]
                if player_season!=player[2]:
                    continue
                else:
                # for i in range(len(player_searched)):
                    if player[1]==player_name:
                        player_searched[k].click()
                        time.sleep(2)
                        break
                    else:
                        continue
                    break
        
        break
            
                

In [40]:
player_name = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li > div > div.info_top > div.name')[0].text
player_searched = driver.find_elements(By.CSS_SELECTOR,'#ulPlayerList > li')
for k in range(len(player_searched)):
    divTag = driver.find_element(By.CSS_SELECTOR,'#ulPlayerList > li.player.player'+str(k)+' > div > div.info_top > div.season > img')
    src = divTag.get_attribute('src')
    regex = re.compile('{}(.*){}'.format(re.escape('season/'), re.escape('.png')))
    player_season = regex.findall(str(src))[0]
    print(player_season)

'#ulPlayerList > li.player.player2.cf'
'#ulPlayerList > li.player.player1.st > div.player_info > div.info_top > div.season'

#print(player_name)
#print(player_season)

ICON
LN
LH
HOT
TKI
TT
NHD


'#ulPlayerList > li.player.player1.st > div.player_info > div.info_top > div.season'

In [49]:
# 제작 중(Run 하면 안됨!)

wrap_teamcolor = '#squadSearchform1 > div > div.search_form > div > div.search_input_detail > div.search_input_detail_teamcolor'
club_teamcolor = ' div > ul > li:nth-child(2)'
nation_teamcolor = 'div > ul > li:nth-child(3)'

for team_color in team_color_top4: # 상위 4개 팀컬러 중에서 순차적으로 선택
    for j in range(11,0,-1):
        position_name = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+' > span > span.info > span.position').text
        PlayersData_sorted=[]
        for playersdata in PlayersData_list: # 선수 데이터 리스트
            if position_name=='LS' or position_name=='RS' or position_name=='ST' or position_name=='LF' or position_name=='RF' or position_name=='CF':
                if playersdata[0]=='ST' or playersdata[0]=='CF':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='LW' or position_name=='RW' or position_name=='LM' or position_name=='RM' or position_name=='LAM' or position_name=='RAM':
                if playersdata[0]=='LW' or playersdata[0]=='RW' or playersdata[0]=='LM' or playersdata[0]=='RM':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='CAM':
                if playersdata[0]=='CAM':
                    PlayersData_sorted.append(playersdata)
            elif position_name == 'LCM' or position_name=='RCM' or position_name=='CM':
                if playersdata[0]=='CM' or playersdata[0]=='CAM':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='LDM' or position_name=='RDM' or position_name=='CDM':
                if playersdata[0]=='CDM' or playersdata[0]=='CM':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='LB' or position_name=='LWB':
                if playersdata[0]=='LB' or playersdata[0]=='LWB':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='RB' or position_name=='RWB':
                if playersdata[0]=='RB' or playersdata[0]=='RWB':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='LCB' or position_name=='RCB' or position_name=='CB':
                if playersdata[0]=='CB':
                    PlayersData_sorted.append(playersdata)
            elif position_name=='GK':
                if playersdata[0]=='GK':
                    PlayersData_sorted.append(playersdata)
            else:
                continue
        squad = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+'> span > span.animate') 
        squad.click() # 스쿼드 십자 모양 클릭
        time.sleep(2)
        search_teamcolor = driver.find_element(By.CSS_SELECTOR, wrap_teamcolor+'>div.wrap_teamcolor')
        if team_color not in total_ClubColorList: # 클럽 팀 컬러에 없다면 국가 팀 컬러이다
            search_teamcolor.click()
            time.sleep(2)
            nation_searching = search_teamcolor.find_element(By.CSS_SELECTOR,nation_teamcolor)
            nation_searching.click()
            time.sleep(2)
            nation_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
            for nation in nation_colorlists:
                if nation.text==team_color:
                    nation.click()
                    time.sleep(2)
                    choose_player()
                else:
                    continue
                break
        else:
            search_teamcolor.click()
            time.sleep(2)
            club_searching = search_teamcolor.find_element(By.CSS_SELECTOR,club_teamcolor)
            club_searching.click()
            time.sleep(2)
            club_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
            for club in club_colorlists:
                if club.text==team_color:
                    club.click()
                    time.sleep(2)
                    choose_player()
                else:
                    continue
                break
        time.sleep(1)
    current_formation = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector.squadmaker-selector > a').text
    save_squad_naming = driver.find_element(By.CSS_SELECTOR,'#squadSaveName').send_keys(team_color+current_formation)
    save_squad_btn = driver.find_element(By.CSS_SELECTOR,'#middle > div > div > div.squad__wrap > div.squad__header-utils > div.squad-save-panel > div.squad-save-name-submit > button.squad-save-panel__btn-save').click()
    alert.accept()

In [55]:
# 실행부분

column_len = driver.find_elements(By.CSS_SELECTOR,'div > div.squadmaker-selector__list__contents > div.squadmaker-selector__list.active > div.squadmaker-selector__list__total>div')
index_len = column_len[0].find_elements(By.CSS_SELECTOR,'ul > li > a')

# print(len(column_len))
# print(len(index_len))

formation_btn = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector')
for i in range(1,len(column_len)+1):
    index_len = column_len[i-1].find_elements(By.CSS_SELECTOR,'ul > li > a')
    for j in range(1,len(index_len)+1):
        time.sleep(2)
        formation_btn.click()
        time.sleep(2)
        formae = formation_btn.find_element(By.CSS_SELECTOR,'div > div.squadmaker-selector__list__contents > div.squadmaker-selector__list.active > div.squadmaker-selector__list__total > div:nth-child('+str(i)+') > ul > li:nth-child('+str(j)+') > a')
        # print(formae)
        if formae.text not in formation_top6: # 지정한 상위 6개 포메이션인지 확인
            continue
        else:    
            time.sleep(2)
            formae.click()
            time.sleep(2)
            wrap_teamcolor = '#squadSearchform1 > div > div.search_form > div > div.search_input_detail > div.search_input_detail_teamcolor'
            club_teamcolor = ' div > ul > li:nth-child(2)'
            nation_teamcolor = 'div > ul > li:nth-child(3)'

            for team_color in team_color_top4: # 상위 4개 팀컬러 중에서 순차적으로 선택
                for j in range(11,0,-1):
                    position_name = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+' > span > span.info > span.position').text
                    PlayersData_sorted=[]
                    for playersdata in PlayersData_list: # 선수 데이터 리스트
                        if position_name=='LS' or position_name=='RS' or position_name=='ST' or position_name=='LF' or position_name=='RF' or position_name=='CF':
                            if playersdata[0]=='ST' or playersdata[0]=='CF':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='LW' or position_name=='RW' or position_name=='LM' or position_name=='RM' or position_name=='LAM' or position_name=='RAM':
                            if playersdata[0]=='LW' or playersdata[0]=='RW' or playersdata[0]=='LM' or playersdata[0]=='RM':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='CAM':
                            if playersdata[0]=='CAM':
                                PlayersData_sorted.append(playersdata)
                        elif position_name == 'LCM' or position_name=='RCM' or position_name=='CM':
                            if playersdata[0]=='CM' or playersdata[0]=='CAM':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='LDM' or position_name=='RDM' or position_name=='CDM':
                            if playersdata[0]=='CDM' or playersdata[0]=='CM':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='LB' or position_name=='LWB':
                            if playersdata[0]=='LB' or playersdata[0]=='LWB':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='RB' or position_name=='RWB':
                            if playersdata[0]=='RB' or playersdata[0]=='RWB':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='LCB' or position_name=='RCB' or position_name=='CB':
                            if playersdata[0]=='CB':
                                PlayersData_sorted.append(playersdata)
                        elif position_name=='GK':
                            if playersdata[0]=='GK':
                                PlayersData_sorted.append(playersdata)
                        else:
                            continue
                    squad = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(j)+'> span > span.animate') 
                    squad.click() # 스쿼드 십자 모양 클릭
                    time.sleep(2)
                    search_teamcolor = driver.find_element(By.CSS_SELECTOR, wrap_teamcolor+'>div.wrap_teamcolor')
                    if team_color not in total_ClubColorList: # 클럽 팀 컬러에 없다면 국가 팀 컬러이다
                        search_teamcolor.click()
                        time.sleep(2)
                        nation_searching = search_teamcolor.find_element(By.CSS_SELECTOR,nation_teamcolor)
                        nation_searching.click()
                        time.sleep(2)
                        nation_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
                        for nation in nation_colorlists:
                            if nation.text==team_color:
                                nation.click()
                                time.sleep(2)
                                choose_player()
                            else:
                                continue
                            break
                    else:
                        search_teamcolor.click()
                        time.sleep(2)
                        club_searching = search_teamcolor.find_element(By.CSS_SELECTOR,club_teamcolor)
                        club_searching.click()
                        time.sleep(2)
                        club_colorlists = driver.find_elements(By.CSS_SELECTOR, wrap_teamcolor+'> div.wrap_teamcolorlist > div > ul > li')
                        for club in club_colorlists:
                            if club.text==team_color:
                                club.click()
                                time.sleep(2)
                                choose_player()
                            else:
                                continue
                            break
                    time.sleep(1)
                current_formation = driver.find_element(By.CSS_SELECTOR,'#squadMaker > div > div.squadmaker__utils > div.squadmaker__utils__aside__selector.squadmaker-selector > a').text
                save_squad_naming = driver.find_element(By.CSS_SELECTOR,'#squadSaveName').send_keys(team_color+current_formation)
                save_squad_btn = driver.find_element(By.CSS_SELECTOR,'#middle > div > div > div.squad__wrap > div.squad__header-utils > div.squad-save-panel > div.squad-save-name-submit > button.squad-save-panel__btn-save').click()
                time.sleep(3)
                alert.accept()
                for d in range(11,0,-1):
                    del_players = driver.find_element(By.CSS_SELECTOR,'#formationPlayer'+str(d)+' > div > div.info.ui-draggable-handle > div.info_utils > a.btn_del').click()
                    time.sleep(1)
                    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#formationPlayer1 > div > div.info.ui-draggable-handle > div.info_utils > a.btn_del"}
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A9C0A3+2212003]
	Ordinal0 [0x00A32CC1+1780929]
	Ordinal0 [0x0094465D+804445]
	Ordinal0 [0x00973475+996469]
	Ordinal0 [0x0097363B+996923]
	Ordinal0 [0x009A1382+1184642]
	Ordinal0 [0x0098EC64+1109092]
	Ordinal0 [0x0099F5B2+1177010]
	Ordinal0 [0x0098EA36+1108534]
	Ordinal0 [0x009683C9+951241]
	Ordinal0 [0x00969396+955286]
	GetHandleVerifier [0x00D49CE2+2746722]
	GetHandleVerifier [0x00D3A234+2682548]
	GetHandleVerifier [0x00B2B34A+524234]
	GetHandleVerifier [0x00B29B60+518112]
	Ordinal0 [0x00A39FBC+1810364]
	Ordinal0 [0x00A3EA28+1829416]
	Ordinal0 [0x00A3EB15+1829653]
	Ordinal0 [0x00A48744+1869636]
	BaseThreadInitThunk [0x76DEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A17B5E+286]
	RtlGetAppContainerNamedObjectP

In [56]:
driver.quit()